In [1]:
import base64
import os
import sys
import math
import logging
from pathlib import Path

import numpy as np
import pandas as pd
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [3]:
playlist = pd.read_csv("../data/interim/recommended_playlist.csv")

In [4]:
playlist.head()

,artist,album,track_x,uri,id,artist_track,track_y,count,date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,track_href,analysis_url,duration_ms,time_signature
0,Maelo Ruiz,En Tiempo de Amor,Te Va a Doler,spotify:track:2h6WIOkTPvHSAD7vR6lMSS,2h6WIOkTPvHSAD7vR6lMSS,Maelo Ruiz: Te Va a Doler,NaN,0.0,NaN,0.610,0.849,8,-4.382,1,0.0461,0.407,0.0000,0.2800,0.742,177.634,audio_features,https://api.spotify.com/v1/tracks/2h6WIOkTPvHS...,https://api.spotify.com/v1/audio-analysis/2h6W...,292773,4
1,Oum Shatt,Gold to Straw,Gold to Straw,spotify:track:2Bo0wCPBd1rApHmb2NNwfe,2Bo0wCPBd1rApHmb2NNwfe,Oum Shatt: Gold to Straw,NaN,0.0,NaN,0.729,0.728,7,-6.689,1,0.0555,0.676,0.2110,0.3400,0.543,175.972,audio_features,https://api.spotify.com/v1/tracks/2Bo0wCPBd1rA...,https://api.spotify.com/v1/audio-analysis/2Bo0...,191796,4
2,Eliza Shaddad,Waters EP,Waters,spotify:track:5FMVdRkv36REATP2FcHOnH,5FMVdRkv36REATP2FcHOnH,Eliza Shaddad: Waters,NaN,0.0,NaN,0.595,0.515,5,-11.595,0,0.0595,0.301,0.0662,0.0971,0.377,128.978,audio_features,https://api.spotify.com/v1/tracks/5FMVdRkv36RE...,https://api.spotify.com/v1/audio-analysis/5FMV...,310139,4
3,Lost Frequencies,Are You with Me (Dash Berlin Remix),Are You with Me - Dash Berlin Radio Edit,spotify:track:2iT0XHIKh3fxEYOnFI4YaS,2iT0XHIKh3fxEYOnFI4YaS,Lost Frequencies: Are You with Me - Dash Berli...,NaN,0.0,NaN,0.532,0.768,8,-3.053,1,0.0547,0.227,0.0000,0.2020,0.647,128.060,audio_features,https://api.spotify.com/v1/tracks/2iT0XHIKh3fx...,https://api.spotify.com/v1/audio-analysis/2iT0...,168095,4
4,Racing Glaciers,Ahead Of You Forever,Moths,spotify:track:7gJ6iD1ktnypMxCRTyLV1f,7gJ6iD1ktnypMxCRTyLV1f,Racing Glaciers: Moths,NaN,0.0,NaN,0.553,0.799,6,-8.755,0,0.0538,0.293,0.2040,0.1370,0.564,127.973,audio_features,https://api.spotify.com/v1/tracks/7gJ6iD1ktnyp...,https://api.spotify.com/v1/audio-analysis/7gJ6...,244425,4


## Create the corresponding Playlist in Spotify

- `playlist_add_items`
- `user_playlist_create`

In [5]:
GDPR_DATA_FOLDER = "../data/raw/MyData/"
client_id_path = "../secrets/client_id.txt"
client_secret_path = "../secrets/client_secret.txt"
user_id_path = "../secrets/user_id.txt"

# TODO: put everything into a single json file

with open(client_id_path, "r") as file:
    client_id = file.read().strip("\n")
    
with open(client_secret_path, "r") as file:
    client_secret = file.read().strip("\n")
    
with open(user_id_path, "r") as file:
    user_id = file.read().strip("\n")

In [6]:
scope = "playlist-modify-public"

In [7]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri="http://example.com",
                                               scope=scope))

In [8]:
playlist_name = "Your Rediscover Past"
description = "Listen to your favorite liked songs you haven't listened to for more than a year"

In [9]:
res = sp.user_playlist_create(user=user_id, name=playlist_name, description=description)

INFO:spotipy.oauth2:User authentication requires interaction with your web browser. Once you enter your credentials and give authorization, you will be redirected to a url.  Paste that url you were directed to to complete the authorization.
INFO:spotipy.oauth2:Opened https://accounts.spotify.com/authorize?client_id=ebf506b727774376b8251703a8ba8b4d&response_type=code&redirect_uri=http%3A%2F%2Fexample.com&scope=playlist-modify-public in your browser


Enter the URL you were redirected to:  http://example.com/?code=AQDKzxzBuNsXqjxVCRclnOIxuZN8sgxaJbJiP0D1-_l8rBnbxTOyL_1xBn9d1O5cTP3Mg3PlixdBuB6HxRKzpnfPCwKpTqCnY4yZcqSIXCfh2UQ1ZOZDb0jXBwMCFMxfiKxAvgrpTDBrVF3eUOhvm0KvsnlUDrcjoWpbpHUB_flS82FQoHBVz2vl7rVKRQ


In [10]:
playlist_id = res["id"]

In [11]:
playlist_id

'5H5m4v4jmcyUfHaA5OUwCJ'

In [12]:
sp.playlist_add_items(playlist_id, playlist["uri"].tolist())

{'snapshot_id': 'Myw1MWM0MGUyZDQ5Y2NmZTg4M2Y4ODE3NmIwY2IwYWViYmVkNGYyNGU2'}

## Change the Playlist Cover Image Acordingly

Before uploading our cover image, we need to refresh our access token.
- https://github.com/plamere/spotipy/issues/749
- https://community.spotify.com/t5/Spotify-for-Developers/Bug-Uploading-cover-image-not-working-with-un-refreshed-access/m-p/5156729#M2056

In [13]:
scope = "ugc-image-upload"

filename = "cover_rediscover_past.jpg"

In [15]:
image = open(f'../img/{filename}', 'rb') #open binary file in read mode
image_read = image.read()
image_64_encode = base64.b64encode(image_read).decode("utf-8") 

In [ ]:
refresh_token = sp.auth_manager.get_cached_token()["refresh_token"]
sp.auth_manager.refresh_access_token(refresh_token)

In [25]:
sp.playlist_upload_cover_image(playlist_id, image_64_encode)